In [ ]:
###파즈베리파이(client)
import io

import socket

import struct

import time

import picamera

 

# Connect a client socket to my_server:8000 (change my_server to the

# hostname of your server)

client_socket = socket.socket()

client_socket.connect(('172.20.10.3', 9000))

 

# Make a file-like object out of the connection

connection = client_socket.makefile('wb')

try:

    with picamera.PiCamera() as camera:

        camera.resolution = (640, 480)

        # Start a preview and let the camera warm up for 2 seconds

        camera.start_preview()

        camera.rotation = 180

        time.sleep(8)

 

        # Note the start time and construct a stream to hold image data

        # temporarily (we could write it directly to connection but in this

        # case we want to find out the size of each capture first to keep

        # our protocol simple)

        start = time.time()

        stream = io.BytesIO()

        for foo in camera.capture_continuous(stream, 'jpeg'):

            # Write the length of the capture to the stream and flush to

            # ensure it actually gets sent

            connection.write(struct.pack('<L', stream.tell()))

            connection.flush()

            # Rewind the stream and send the image data over the wire

            stream.seek(0)

            connection.write(stream.read())

            # If we've been capturing for more than 30 seconds, quit

            if time.time() - start > 30:

                break

            # Reset the stream for the next capture

            stream.seek(0)

            stream.truncate()

    # Write a length of zero to the stream to signal we're done

    connection.write(struct.pack('<L', 0))

finally:

    connection.close()

    client_socket.close()

In [ ]:
###PC(server)
import io
import socket
import struct
import cv2
from PIL import Image

 
server_socket = socket.socket()
server_socket.bind(('0.0.0.0', 9000))
server_socket.listen(0)
 
# Accept a single connection and make a file-like object out of it
connection = server_socket.accept()[0].makefile('rb')
try:
    while True:
        # Read the length of the image as a 32-bit unsigned int. If the
        # length is zero, quit the loop
        image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
        if not image_len:
            break
        # Construct a stream to hold the image data and read the image
        # data from the connection
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        # Rewind the stream, open it as an image with PIL and do some
        # processing on it
        image_stream.seek(0)
        image = Image.open(image_stream)
        image.show()
        image.save('mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm.jpg')
        print('Image is %dx%d' % image.size)
        image.verify()
        print('Image is verified')
finally:
    connection.close()
    server_socket.close()